# Data Source

> Data Source functions and classes

In [ ]:
#| default_exp data_source

In [ ]:
#| hide
from nbdev.showdoc import *
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
from emb_opt.imports import *
from emb_opt.utils import build_batch_from_embeddings
from emb_opt.module import Module
from emb_opt.schemas import Item, Query, Batch, DataSourceFunction, DataSourceResponse

/home/dmai/miniconda3/envs/emb_opt/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The Data Source step runs a set of queries against some data source.

The query is defined by the `DataSourceFunction` schema, which maps inputs `List[Query]` to outputs `List[DataSourceResponse]`.

The `DataSourceModule` manages execution of a `DataSourceFunction`. The `DataSourceModule` gathers valid queries, sends them to the `DataSourceFunction`, and processes the results.

In [ ]:
#| export

class DataSourceModule(Module):
    def __init__(self, 
                 function: DataSourceFunction # data function
                ):
        super().__init__(DataSourceResponse, function)
        
    def gather_inputs(self, batch: Batch) -> (List[Tuple], List[Query]):
        idxs, inputs = batch.flatten_queries()
        return (idxs, inputs)
    
    def scatter_results(self, batch: Batch, idxs: List[Tuple], results: List[DataSourceResponse]):
        for (q_idx, r_idx), result in zip(idxs, results):
            batch_item = batch.get_item(q_idx, r_idx)
            if result.data:
                batch_item.data.update(result.data)
                
            if not result.valid:
                batch_item.update_internal(removed=True, removal_reason='invalid query')
                
            elif len(result.query_results)==0:
                batch_item.update_internal(removed=True, removal_reason='query returned no results')
                
            else:
                batch_item.add_query_results(result.query_results)

In [ ]:
def build_batch():
    embeddings = [[0.1], [0.2], [0.3]]
    batch = build_batch_from_embeddings(embeddings)
    return batch

def data_source_test(queries: List[Query]) -> List[DataSourceResponse]:
    results = []
    for i, query in enumerate(queries):
        if i==0:
            response = DataSourceResponse(valid=False, data={'test':'test false response'},
                                         query_results=[Item.from_minimal(item='', embedding=[0.1])])
        elif i==1:
            response = DataSourceResponse(valid=True, data={'test':'test empty response'},
                                         query_results=[])
        elif i==2:
            response = DataSourceResponse(valid=True, data={'test':'test normal response'},
                                         query_results=[Item.from_minimal(item='1', embedding=[0.1]), 
                                                       Item.from_minimal(item='2', embedding=[0.2])])
        results.append(response)
    return results

batch = build_batch()
data_module = DataSourceModule(data_source_test)
batch2 = data_module(batch)
assert [i.internal.removed for i in batch2] == [True, True, False]

for q in batch2:
    for r in q:
        assert r.internal.parent_id == q.id

Given pydantic data parsing, the function can also return a json response

In [ ]:
def json_test(queries: List[Query]) -> List[Dict]:
    results = [
        {
            'valid' : True,
            'data' : {},
            'query_results' : [
                {
                    'id' : None,
                    'item' : 'test',
                    'embedding' : [0.1],
                    'score' : None,
                    'data' : None
                }
            ]
        }
        for i in queries
    ]
    return results

batch = build_batch()
data_module = DataSourceModule(json_test)
batch2 = data_module(batch)

In [ ]:
#| export

class DataSourcePlugin():
    '''
    DataSourcePlugin - documentation for plugin functions to `DataSourceFunction`
    
    A valid `DataSourceFunction` is any function that maps `List[Query]` to 
    `List[DataSourceResponse]`. The inputs will be given as `Query` objects. 
    The outputs can be either a list of `DataSourceResponse` objects or a list of 
    valid json dictionaries that match the `DataSourceResponse` schema
    
    Query schema:
    
    `{
        'item' : Optional[Any],
        'embedding' : List[float],
        'data' : Optional[Dict],
        'query_results': [] # will be empty at this stage
    }`
    
    Item schema:
    
    `{
        'id' : Optional[Union[str, int]]
        'item' : Optional[Any],
        'embedding' : List[float],
        'score' : None, # will be None at this stage
        'data' : Optional[Dict],
    }`
    
    Input schema:
    
    `List[Query]`
    
    DataSourceResponse schema:
    
    `{
        'valid' : bool,
        'data' : Optional[Dict],
        'query_results' : List[Item]
    }`
    
    Output schema:
    
    `List[DataSourceResponse]`
    
    '''
    def __call__(self, inputs: List[Query]) -> List[DataSourceResponse]:
        pass

The `NumpyDataPlugin` data source works with any numpy array of embeddings

In [ ]:
#| export

class NumpyDataPlugin(DataSourcePlugin):
    '''
    NumpyDataPlugin - data plugin for working with numpy arrays. The 
    data query will run `k` nearest neighbors of the query embeddings 
    against the `item_embeddings` using `distance_metric`
    
    Optionally, `item_data` can be provided as a list of dicts, where 
    `item_data[i]` corresponds to the data for `item_embeddings[i]`.
    
    If `item_data` is provided, `item_data[i]['id_key']` defines 
    the ID of item `i`, and `item_data[i]['item_key']` defines the 
    specific item `i`
    
    `distance_metric` is any valid scipy distance metric. see 
    https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cdist.html
    '''
    def __init__(self,
                 k: int,                               # k nearest neighbors to return
                 item_embeddings: np.ndarray,          # item embeddings
                 item_data: Optional[List[Dict]]=None, # Optional dict of item data
                 id_key: Optional[str]=None,           # Optional key for item id (should be in `item_data` dict)
                 item_key: Optional[str]=None,         # Optional key for item value (should be in `item_data` dict)
                 distance_metric: str='euclidean'      # distance metric, see options at https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cdist.html
                ):
        
        self.k = k
        self.item_embeddings = item_embeddings
        self.item_data = item_data
        self.id_key = id_key
        self.item_key = item_key
        self.distance_metric = distance_metric
        
    def __call__(self, inputs: List[Query]) -> List[DataSourceResponse]:
        
        queries = np.array([i.embedding for i in inputs])
        distances = cdist(queries, self.item_embeddings, metric=self.distance_metric)
        topk = distances.argsort(-1)[:, :self.k]
        
        outputs = []
        for i in range(len(inputs)):
            items = []
            query_data = {'query_distance' : []}
            for j in topk[i]:
                query_data['query_distance'].append(distances[i,j])
                
                data = None
                item_value = None
                item_id = None
                
                if self.item_data:
                    data = dict(self.item_data[j])
                    if self.id_key:
                        item_id = data.pop(self.id_key)
                        
                    if self.item_key:
                        item_value = data.pop(self.item_key)
                                
                item = Item(id=item_id, 
                            item=item_value,
                            embedding=self.item_embeddings[j], 
                            data=data, 
                            score=None)
                items.append(item)
                
            result = DataSourceResponse(valid=True, data=query_data, query_results=items)
            outputs.append(result)
            
        return outputs

In [ ]:
n_vectors = 256
d_vectors = 64
k = 10
n_queries = 5

vectors = np.random.randn(n_vectors, d_vectors)
vector_data = [{'index':str(np.random.randint(0,1e6)), 
                'other':np.random.randint(0,1e3), 
                'item':str(np.random.randint(0,1e4))} 
               for i in range(vectors.shape[0])]

data_function = NumpyDataPlugin(k, vectors, vector_data, id_key='index', item_key='item', distance_metric='cosine')
data_module = DataSourceModule(data_function)

batch = build_batch_from_embeddings(np.random.randn(n_queries, d_vectors))
batch2 = data_module(batch)

for q in batch2:
    for r in q:
        assert r.internal.parent_id == q.id